In [ ]:
import os
import re
import PyPDF2
from PyPDF2 import PdfWriter, PdfReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
import urllib.request
import tempfile

def extract_numbers(text):
    return [int(x) for x in re.findall(r'\d+', text) if int(x) != 100]

def extract_total_mark(text):
    match = re.search(r'= (\d+) / 100', text)
    if match:
        return int(match.group(1))
    return None

def extract_annotations(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        extracted_annotations = []

        for page in reader.pages:
            if '/Annots' in page:
                annotations = page['/Annots']

                for annot in annotations:
                    annot_object = annot.get_object()
                    annot_content = annot_object.get('/Contents')
                    if annot_content:
                        extracted_annotations.append(annot_content)

    return extracted_annotations

def add_checkmark_overlay(pdf_path, checkmark_url):
    # Download the checkmark image to a temporary file
    with tempfile.NamedTemporaryFile(delete=False) as temp_checkmark:
        urllib.request.urlretrieve(checkmark_url, temp_checkmark.name)
        checkmark_path = temp_checkmark.name

        # Create a new PDF with the checkmark overlay
        with open(pdf_path, 'rb') as f:
            original = PdfReader(f)

            # Create a new PDF canvas
            buffer = BytesIO()
            c = canvas.Canvas(buffer, pagesize=letter)

            # Draw the checkmark on the canvas
            c.drawImage(checkmark_path, 50, 750, width=20, height=20)
            c.showPage()
            c.save()

            # Merge the checkmark overlay with the original PDF
            buffer.seek(0)
            overlay = PdfReader(buffer)

            writer = PdfWriter()

            for i, page in enumerate(original.pages):
                if i == 0:
                    page.merge_page(overlay.pages[0])
                writer.add_page(page)

            output_filename = os.path.splitext(pdf_path)[0] + '_checked.pdf'
            with open(output_filename, 'wb') as output:
                writer.write(output)


def check_total_marks(pdf_path):
    annotations = extract_annotations(pdf_path)

    text = ' '.join(annotations)

    total_mark = extract_total_mark(text)
    if not total_mark:
        print(f"Error in {pdf_path}: Total mark not found.")
        return

    numbers = extract_numbers(text)
    numbers.remove(total_mark)

    calculated_total_mark = sum(numbers)

    if calculated_total_mark == total_mark:
        checkmark_url = 'https://tinyurl.com/2bbg4224'
        add_checkmark_overlay(pdf_path, checkmark_url)
    else:
        print(f"Checked {pdf_path}: Total marks do not match! {calculated_total_mark} != {total_mark}")
        print(f"Numbers extracted from {pdf_path}: {numbers}")

scripts_folder = os.getcwd()

for filename in os.listdir(scripts_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(scripts_folder, filename)
        check_total_marks(pdf_path)
